In [49]:
from torchtext.data import Field, TabularDataset, BucketIterator,LabelField
import torch.nn as nn
import random
import torch

In [105]:
path = '/Users/babyhandzzz/Desktop/ELEPH@NT/Datasets/clean_IMDB.csv'

In [106]:
# Load the data
tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize,batch_first=True)
LABEL = LabelField(dtype=torch.int,batch_first=True)
fields = [('text', TEXT), ('label',LABEL)]
training_data=TabularDataset(path=path,format='csv',fields = fields, skip_header=True)

In [109]:
# train test split
SEED=2020
train_data, valid_data = training_data.split(split_ratio=0.7, random_state=random.seed(SEED))

In [110]:
# vectorize words GloVe
TEXT.build_vocab(train_data,min_freq=3,vectors="glove.6B.100d")  
LABEL.build_vocab(train_data)

In [111]:
TEXT.vocab

In [98]:
train_iterator, valid_iterator = BucketIterator.splits((train_data, valid_data), batch_size = 32,
sort_key = lambda x: len(x.text),sort_within_batch=True)

In [89]:
class LSTMClassifier(nn.Module):
	def __init__(self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
		super(LSTMClassifier, self).__init__()
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		self.vocab_size = vocab_size
		self.embedding_length = embedding_length
		
		self.word_embeddings = nn.Embedding(vocab_size, embedding_length)
		self.word_embeddings.weight = nn.Parameter(weights, requires_grad=False)
		self.lstm = nn.LSTM(embedding_length, hidden_size)
		self.label = nn.Linear(hidden_size, output_size)
		
	def forward(self, input_embedding, batch_size=None):
	
		input_ = self.word_embeddings(input_embedding) 
		return input_ 


In [99]:
batch_size = 32
output_size = 1
hidden_size = 150
vocab_size = len(TEXT.vocab)
embedding_length = 100
word_embeddings = TEXT.vocab.vectors

In [91]:
model = LSTMClassifier(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)

In [92]:
for idx, batch in enumerate(train_iterator):
    text = batch.text
    target = batch.label
    prediction = model(text)
    print(prediction)
    break

tensor([[[-9.4820e-01,  2.9996e-01, -2.2521e+00,  ...,  9.8973e-01,
           1.4729e+00,  1.7758e+00],
         [ 1.0750e-01,  6.0153e-01,  2.2525e-01,  ...,  9.6780e-01,
          -7.4871e-01, -2.7538e-02],
         [ 1.4004e-01, -1.0123e+00,  5.8283e-01,  ..., -5.5197e-01,
          -3.0484e-01,  1.4219e-01],
         ...,
         [-3.7835e-01,  9.1582e-01,  6.2399e-01,  ..., -2.1765e+00,
           4.2013e-01,  5.9320e-01],
         [ 6.4673e-01,  2.8239e-01, -1.9636e+00,  ..., -6.2310e-02,
           1.2249e+00, -6.6461e-01],
         [ 2.4890e-01, -2.4176e-01,  4.3373e-01,  ...,  4.6947e-01,
           6.6965e-01,  5.4410e-01]],

        [[ 3.9167e-01, -2.3850e-01, -1.0974e+00,  ...,  5.6394e-01,
           5.6635e-01, -3.1735e-01],
         [ 1.1163e+00, -4.5154e-01,  1.3764e+00,  ...,  7.1564e-01,
          -8.4829e-01, -9.3492e-01],
         [ 1.5986e-01,  4.4215e-01, -7.0887e-01,  ..., -7.2369e-01,
           9.7959e-01,  1.1549e+00],
         ...,
         [ 1.0949e+00,  2

In [100]:
TEXT.vocab.vectors

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0629, -0.5078, -0.5660,  ...,  0.0900,  0.3337, -0.3578],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])